In [ ]:
!pip install transformers
!pip install nltk
!pip install textattack

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
import nltk
import random
import pandas as pd
from nltk.corpus import wordnet
print("GPU is available: ", tf.config.experimental.list_physical_devices("GPU"))

GPU is available:  []


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# from transformers import pipeline

# # Initialize the translation pipeline
# translator = pipeline("translation", model="Helsinki-NLP/opus-mt-en-fr")

# # Original text
# original_text = "Since the windshield exerts a force on the mosquito, which we can call action, the mosquito exerts an equal and opposite force on the windshield, called the reaction."

# # Translate to French
# translated_text = translator(original_text, max_length=400)[0]['translation_text']

# # Translate back to English
# back_translated_text = translator(translated_text, max_length=40)[0]['translation_text']

# # Collect augmented data
# augmented_data = [original_text, back_translated_text]

# # You can add these to your dataset for data augmentation
# print(augmented_data)


Importing the dataset

In [ ]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet
import random

def synonym_replacement(text):
    words = text.split()
    new_words = []
    for word in words:
        synsets = wordnet.synsets(word)
        if synsets:
            synonyms = [lemma.name() for synset in synsets for lemma in synset.lemmas()]
            if synonyms:
                new_word = random.choice(synonyms)
                new_words.append(new_word)
            else:
                new_words.append(word)
        else:
            new_words.append(word)
    return ' '.join(new_words)
original_text = "Since the windshield exerts a force on the mosquito, which we can call action, the mosquito exerts an equal and opposite force on the windshield, called the reaction.."
augmented_text = synonym_replacement(original_text)
print(augmented_text)



In [ ]:
import xml.etree.ElementTree as ET
import pandas as pd

def get_data_from_xml_child(child):
    Row = {}
    Row["StudentID"] = child.find("MetaInfo").get('StudentID')
    Row["TaskID"] = child.find("MetaInfo").get('TaskID')
    Row["DataSource"] = child.find("MetaInfo").get('DataSource')
    Row["ProblemDescription"] = child.find("ProblemDescription").text
    Row["Question"] = child.find("Question").text
    Row["Answer"] = child.find("Answer").text
    Row["Annotation_label"] = child.find("Annotation").get("Label")
    Row["Annotation_ContextRequired"] = child.find("Annotation").find("AdditionalAnnotation").get('ContextRequired')
    Row["Annotation_ExtraInfoInAnswer"] = child.find("Annotation").find("AdditionalAnnotation").get('ExtraInfoInAnswer')
    Row["Annotation_comments"] = child.find("Annotation").find("Comments").text
    Row["Annotation_comments_watch"] = child.find("Annotation").find("Comments").get("Watch")
    Row["ReferenceAnswers"] = child.find("ReferenceAnswers").text
    return Row

# xml_data = open('/kaggle/input/asag-dt-grad-xml/grade_data.xml', 'r').read()  # Read file
# Read the file from your Google Drive
with open('/content/drive/MyDrive/Colab Notebooks/grade_data.xml', 'r') as f:
    xml_data = f.read()
root = ET.XML(xml_data)  # Parse XML

data = []
cols = []
for i, child in enumerate(root):
    data.append(get_data_from_xml_child(child))

In [ ]:
df = pd.DataFrame(data)
df.head()

In [ ]:
df.columns

In [ ]:
df_draft = df[['ProblemDescription', 'Question',
       'Answer', 'Annotation_ContextRequired', 'Annotation_ExtraInfoInAnswer',
       'Annotation_comments', 'Annotation_comments_watch', 'ReferenceAnswers',"Annotation_label"]]
df_draft.columns = ['PD', 'Q','A', 'ACR', 'AEI','AC', 'ACW', 'RA',"target"]

In [ ]:
df_draft.head()

In [ ]:
def change_label(txt):
    rs = ""
    splits = str(txt).split("|")


    for sp in splits:
        #print(sp)
        if "1" in sp:
            #print("in")
            rs = rs+sp+" "

    return rs.replace("(1)","").strip()

In [ ]:
df_draft["target"] = df_draft["target"].apply(change_label)

In [ ]:
df_draft["target"].value_counts()

In [ ]:
df1 = df_draft[["A","RA","target","ACR","AEI","ACW"]]
df1.head()

In [ ]:
#  df_draft["text"] = df_draft["A"]+" "+df_draft["RA"]
# df_draft["text"] = df_draft["A"]+" "+df_draft["RA"]

In [ ]:
# df_draft["text"] = df_draft["Q"]+" "+df_darft["A"]+" "+df_darft["RA"]

In [ ]:
#$ df1 = df_draft[["text","target","ACR","AEI","ACW"]]

In [ ]:
# len(df1["text"][55])

In [ ]:
def transfrom_class(txt):
    txt = txt.strip()
    if "correct" == txt:
        return 0
    elif "incorrect" == txt:
        return 1
    elif "correct_but_incomplete" == txt:
        return 2
    elif "contradictory" == txt:
        return 3
    elif "correct_but_incomplete incorrect" == txt:
        return 4

In [ ]:
df1["target"].apply(transfrom_class).value_counts()

In [ ]:
df1["target"] = df1["target"].apply(transfrom_class)

In [ ]:
df1.head()

In [ ]:
df1 = df1.drop(df1[df1["target"] == 4].index)

In [ ]:
df1["target"].value_counts()

In [ ]:
import spacy
from sklearn.feature_extraction import _stop_words
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
import string

In [ ]:
nlp = spacy.load('en_core_web_sm')

stopwords = _stop_words.ENGLISH_STOP_WORDS
lemmatizer = WordNetLemmatizer()

def clean(doc):
    #text_no_namedentities = []
    #document = nlp(doc)
    #ents = [e.text for e in document.ents]
    #for item in document:
    #    if item.text in ents:
    #        pass
    #    else:
    #        text_no_namedentities.append(item.text)

    #doc = (" ".join(text_no_namedentities))

    doc = doc.lower().strip()
    doc = doc.replace("</br>", " ")
    doc = doc.replace("-", " ")

    doc = "".join([char for char in doc if char not in string.punctuation and not char.isdigit()])
    doc = " ".join([token for token in doc.split() if token not in stopwords])
    doc = "".join([lemmatizer.lemmatize(word) for word in doc])
    return doc

In [ ]:
df1.A.iloc[0]
df1.RA.iloc[0]


In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
clean(df1.A.iloc[0])
clean(df1.RA.iloc[0])

In [ ]:
df1["A"] = df1.A.apply(clean)
df1["RA"] = df1.RA.apply(clean)

In [ ]:
df1.head()

In [ ]:
# df1 = pd.read_csv("/kaggle/input/asag-dt-grad/clean_data_4_labels.csv")

In [ ]:
print(df1.columns)
# del df1["Unnamed: 0"]

In [ ]:
df1.head()

In [ ]:
df = df1[["A","RA","target"]]

from heere started the augmented sbert stratigie  

---

the Augmented SBERT (In-domain) strategy

In [ ]:
pip install sentence-transformers



```
# This is formatted as code
```

## *Train a Cross-Encoder*

In [ ]:
from sentence_transformers import SentenceTransformer, InputExample
from sentence_transformers.cross_encoder import CrossEncoder
from torch.utils.data import DataLoader
import numpy as np

# Initialize a cross-encoder
cross_encoder = CrossEncoder('bert-base-uncased', num_labels=len(df['target'].unique()))

# Prepare the data
gold_examples = [InputExample(texts=[row['A'], row['RA']], label=row['target']) for index, row in df.iterrows()]

# Convert to DataLoader
train_dataloader = DataLoader(gold_examples, shuffle=True, batch_size=16)

# Train the cross-encoder
cross_encoder.fit(train_dataloader, epochs=1, warmup_steps=100)


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/57 [00:00<?, ?it/s]

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Initialize the T5 model and tokenizer
model_name = 't5-small'  # You can choose a larger model for better results
t5_model = T5ForConditionalGeneration.from_pretrained(model_name).to('cuda')
t5_tokenizer = T5Tokenizer.from_pretrained(model_name)

def generate_paraphrases(sentence, num_paraphrases=1):
    # T5 uses a task-specific prefix, e.g., "paraphrase: "
    text = f"paraphrase: {sentence} </s>"
    encoding = t5_tokenizer.encode_plus(text, padding=True, return_tensors="pt")
    input_ids, attention_masks = encoding["input_ids"].to('cuda'), encoding["attention_mask"].to('cuda')

    # Generate paraphrases
    outputs = t5_model.generate(
        input_ids=input_ids,
        attention_mask=attention_masks,
        max_length=256,
        num_return_sequences=num_paraphrases,
        num_beams=10
    )

    # Decode paraphrases
    paraphrases = [t5_tokenizer.decode(output, skip_special_tokens=True, clean_up_tokenization_spaces=True) for output in outputs]
    return paraphrases

# Use the function to generate paraphrases for each answer and reference answer
augmented_examples = []
for _, row in df.iterrows():
    paraphrased_As = generate_paraphrases(row['A'])
    paraphrased_RAs = row['RA']
    for paraphrased_A in paraphrased_As:
        for paraphrased_RA in paraphrased_RAs:
            augmented_examples.append(InputExample(texts=[paraphrased_A, paraphrased_RA], label=row['target']))


In [ ]:
pip install nltk bert_score sentence-transformers

In [ ]:
from nltk.translate.bleu_score import sentence_bleu
from bert_score import score
from sentence_transformers import SentenceTransformer, util

# Load a semantic similarity model
sim_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

def calculate_similarity(original, paraphrase):
    original_emb = sim_model.encode(original, convert_to_tensor=True)
    paraphrase_emb = sim_model.encode(paraphrase, convert_to_tensor=True)
    cosine_scores = util.pytorch_cos_sim(original_emb, paraphrase_emb)
    return cosine_scores.item()

# Perform checks
for example in augmented_examples:
    original = example.texts[0]  # Assuming the first text is the original
    paraphrase = example.texts[1]  # Assuming the second text is the paraphrase

    print(f"Original: {original}")
    print(f"Paraphrase: {paraphrase}")

    # Check semantic similarity
    similarity = calculate_similarity(original, paraphrase)
    print(f"Semantic Similarity: {similarity}")

    # Calculate BLEU score for diversity (inverted interpretation)
    reference = [original.split()]
    candidate = paraphrase.split()
    bleu = sentence_bleu(reference, candidate)
    print(f"BLEU score (lower indicates more diversity): {bleu}")

    # Calculate BERTScore
    P, R, F1 = score([paraphrase], [original], lang="en", verbose=False)
    print(f"BERTScore Precision: {P.item()}")
    print(f"BERTScore Recall: {R.item()}")
    print(f"BERTScore F1: {F1.item()}")
    print("\n")


Train a Bi-Encoder (SBERT)

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from sentence_transformers import models, SentenceTransformer, losses
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

# Define the SentenceTransformer model with RoBERTa
word_embedding_model = models.Transformer('roberta-base')
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
bi_encoder = SentenceTransformer(modules=[word_embedding_model, pooling_model])

# Combine gold and silver data
train_examples = gold_examples + augmented_examples  # Make sure these are defined

# Train the model
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
train_loss = losses.SoftmaxLoss(
    model=bi_encoder,
    sentence_embedding_dimension=bi_encoder.get_sentence_embedding_dimension(),
    num_labels=len(df['target'].unique())
)

# Fit the model
bi_encoder.fit(train_objectives=[(train_dataloader, train_loss)], epochs=4)


In [ ]:
def batch_to_text(input_ids_batch, tokenizer):
    texts = []
    for input_ids in input_ids_batch:
        # Remove padding token ids and convert to text
        text = tokenizer.decode(input_ids, skip_special_tokens=True)
        texts.append(text)
    return texts


In [ ]:


# Define a classification head module
class ClassificationHead(nn.Module):
    def __init__(self, embedding_dim, num_classes):
        super(ClassificationHead, self).__init__()
        self.linear = nn.Linear(embedding_dim, num_classes)

    def forward(self, features):
        logits = self.linear(features)
        return logits

# Assuming X_val and y_val are your validation datasets containing the input texts and labels
X = df[['A', 'RA']]
y = df['target']

# Stratified split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)

# Tokenize the validation dataset
val_encodings = bi_encoder.tokenizer(X_val['A'].tolist(), X_val['RA'].tolist(), padding=True, truncation=True, return_tensors="pt")

# Create a Dataset object from the tokenized texts and labels
val_dataset = TensorDataset(val_encodings['input_ids'], val_encodings['attention_mask'], torch.tensor(y_val.tolist()))

# Create the DataLoader from the Dataset object
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Initialize the classification head
embedding_dim = bi_encoder.get_sentence_embedding_dimension()
num_classes = len(df['target'].unique())
classifier = ClassificationHead(embedding_dim, num_classes)

# Move the classifier to the same device as the SentenceTransformer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
classifier.to(device)
bi_encoder.to(device)

# Evaluate the combined model (SentenceTransformer + classification head)
bi_encoder.eval()  # Set the SentenceTransformer to evaluation mode
classifier.eval()  # Set the classification head to evaluation mode

true_labels = []
predictions = []

with torch.no_grad():
    for batch in val_dataloader:
        input_ids, attention_mask, labels = batch
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)

        # Convert batch to text
        texts = batch_to_text(input_ids.cpu(), bi_encoder.tokenizer)

        # Generate embeddings
        embeddings = bi_encoder.encode(texts, convert_to_tensor=True, show_progress_bar=False)
        embeddings = embeddings.to(device)

        # Get the logits from the classifier
        logits = classifier(embeddings)

        # Get predictions from the logits
        preds = torch.argmax(logits, dim=1)

        # Collect the true labels and predictions for evaluation
        true_labels.extend(labels.cpu().numpy())
        predictions.extend(preds.cpu().numpy())

# Compute the accuracy and F1 score
accuracy = accuracy_score(true_labels, predictions)
f1_macro = f1_score(true_labels, predictions, average='macro')
f1_weighted = f1_score(true_labels, predictions, average='weighted')

### Prepare the Validation *Data*

### Calculate Accuracy and F1 Score

In [ ]:
accuracy

In [ ]:
f1_macro

In [ ]:
f1_weighted

In [ ]:
pip install sentence-transformers

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
from sentence_transformers import SentenceTransformer, InputExample
# Initialize the T5 model and tokenizer
model_name = 't5-small'  # You can choose a larger model for better results
t5_model = T5ForConditionalGeneration.from_pretrained(model_name).to('cuda')
t5_tokenizer = T5Tokenizer.from_pretrained(model_name)

def generate_paraphrases(sentence, num_paraphrases=1):
    # T5 uses a task-specific prefix, e.g., "paraphrase: "
    text = f"paraphrase: {sentence} </s>"
    encoding = t5_tokenizer.encode_plus(text, padding=True, return_tensors="pt")
    input_ids, attention_masks = encoding["input_ids"].to('cuda'), encoding["attention_mask"].to('cuda')

    # Generate paraphrases
    outputs = t5_model.generate(
        input_ids=input_ids,
        attention_mask=attention_masks,
        max_length=256,
        num_return_sequences=num_paraphrases,
        num_beams=10
    )

    # Decode paraphrases
    paraphrases = [t5_tokenizer.decode(output, skip_special_tokens=True, clean_up_tokenization_spaces=True) for output in outputs]
    return paraphrases

# Use the function to generate paraphrases for each answer and reference answer
augmented_examples = []
for _, row in df.iterrows():
    paraphrased_As = generate_paraphrases(row['A'])
    paraphrased_RAs = row['RA']
    for paraphrased_A in paraphrased_As:
        for paraphrased_RA in paraphrased_RAs:
            augmented_examples.append(InputExample(texts=[paraphrased_A, paraphrased_RA], label=row['target']))


In [ ]:
paraphrased_As

NameError: ignored

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load T5 model and tokenizer
t5_model = T5ForConditionalGeneration.from_pretrained("t5-small")
tokenizer = T5Tokenizer.from_pretrained("t5-small")

# Function for text augmentation using T5 model
def t5_augment(text):
    input_text = text
    input_ids = tokenizer.encode(input_text, return_tensors="pt")

    # Generate augmented text
    output_ids = t5_model.generate(input_ids, max_length=50, num_beams=5, length_penalty=1.0, early_stopping=True)
    augmented_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    return augmented_text

# Apply the T5-based data augmentation function to the "A" column
# df1['Augmented_A'] = df1['A'].apply(t5_augment)
new_df = pd.DataFrame({
    'A': df1['A'].apply(t5_augment),
    'RA': df1['RA'],
    'target': df1['target']
})
# # Create a new DataFrame with the augmented "A" and the original "RA"
# new_df = pd.DataFrame({'A': df1['Augmented_A'], 'RA': df1['RA'], 'target': df1['target']})
# # Reset the index for the new DataFrame
# new_df = new_df.reset_index(drop=True)
# new_df.head()

In [ ]:
new_df.describe(include='all')

In [ ]:
combined_df = pd.concat([new_df, df], ignore_index=True)

In [ ]:
combined_df.head()


In [ ]:
combined_df["A"].value_counts()

In [ ]:
df = combined_df[["A","RA","target"]]
combined_df["RA"].value_counts()

In [ ]:
batch_1 = df


In [ ]:
batch_1.head()

In [ ]:
batch_1["target"].value_counts()

In [ ]:
batch_1.isnull().sum()
batch_1=batch_1.dropna(axis=0)

In [ ]:
batch_1.describe(include='all')

In [ ]:
# For DistilBERT:
# model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
# model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
# tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
# model = model_class.from_pretrained(pretrained_weights)
from transformers import RobertaTokenizer, RobertaForSequenceClassification

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base')

Model #1: Preparing the Dataset

Right now, the variable `model` holds a pretrained distilBERT model -- a version of BERT that is smaller, but much faster and requiring a lot less memory.

## Model #1: Preparing the Dataset
Before we can hand our sentences to BERT, we need to do some minimal processing to put them in the format it requires.

### Tokenization
Our first step is to tokenize the sentences -- break them up into word and subwords in the format BERT is comfortable with.

In [ ]:
inputs = [(a, ra) for a, ra in zip(batch_1["A"], batch_1["RA"])]


In [ ]:
type(inputs)

In [ ]:
inputs = [(a, ra) for a, ra in zip(batch_1["A"], batch_1["RA"])]
# tokenizer(, , return_tensors='pt')
encoded_inputs = tokenizer(inputs,
                           padding=True,
                           truncation=True,
                           return_tensors='pt')
encoded_inputs

In [ ]:
# tokenized = batch_1["clean_text"].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
batch_1.reset_index(drop=True, inplace=True)
tokenized = []
for i in range(len(batch_1["A"])):
    # Tokenize the sentence pair
    encoded = tokenizer.encode_plus(batch_1["A"][i], batch_1["RA"][i],
                                     add_special_tokens=True,
                                     max_length=512,
                                     truncation=True,
                                     padding='max_length',
                                     return_attention_mask=True,
                                     return_token_type_ids=True)
    # Add the encoded sequence to the list
    tokenized.append(encoded)

In [ ]:
# max_len = 0
# for i in tokenized.values:
#     if len(i) > max_len:
#         max_len = len(i)
max_len = 0
for i in tokenized:
    if len(i) > max_len:
        max_len = len(i)
padded = np.array([enc['input_ids'] + [0]*(max_len-len(enc['input_ids'])) for enc in tokenized])



In [ ]:
padded

In [ ]:
np.array(padded).shape

### Masking
If we directly send `padded` to BERT, that would slightly confuse it. We need to create another variable to tell it to ignore (mask) the padding we've added when it's processing its input. That's what attention_mask is:

In [ ]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

In [ ]:
# input_ids = torch.tensor(padded)
# attention_mask = torch.tensor(attention_mask)

# with torch.no_grad():
#     last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
# input_ids = torch.tensor(padded)
# attention_mask = torch.tensor(attention_mask)
# batch_size = 16
# num_samples = len(input_ids)
# last_hidden_states = []

# with torch.no_grad():
#     for i in range(0, num_samples, batch_size):
#         input_ids_batch = torch.tensor(padded[i:i+batch_size])
#         attention_mask_batch = torch.tensor(attention_mask[i:i+batch_size])
#         batch_last_hidden_states = model(input_ids_batch, attention_mask=attention_mask_batch)
#         last_hidden_states.append(batch_last_hidden_states.last_hidden_state)

# last_hidden_states = torch.cat(last_hidden_states, dim=0)
input_ids = torch.tensor(padded)
attention_mask = torch.tensor(attention_mask)
batch_size = 16
num_samples = len(input_ids)
last_hidden_states = []

with torch.no_grad():
    for i in range(0, num_samples, batch_size):
        input_ids_batch = torch.tensor(padded[i:i+batch_size])
        attention_mask_batch = torch.tensor(attention_mask[i:i+batch_size])
        batch_last_hidden_states = model(input_ids_batch, attention_mask=attention_mask_batch)
        last_hidden_states.append(batch_last_hidden_states.logits)

last_hidden_states = torch.cat(last_hidden_states, dim=0)


In [ ]:
if len(last_hidden_states.shape) == 2:
    features = last_hidden_states.numpy()
else:
    features = last_hidden_states[:, 0, :].numpy()

#  features = last_hidden_states[:, 0, :].numpy()
#  features = last_hidden_states[:, :].numpy()
# last_hidden_states
len(last_hidden_states.shape)

2

In [ ]:
labels = batch_1["target"]
print('test print here ')

test print here 


In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)
train_features

array([[0.07821109, 0.08993697],
       [0.08524515, 0.09736826],
       [0.08686496, 0.07779203],
       ...,
       [0.10030397, 0.06955218],
       [0.10056917, 0.07043003],
       [0.08725807, 0.08558302]], dtype=float32)

We now train the LogisticRegression model. If you've chosen to do the gridsearch, you can plug the value of C into the model declaration (e.g. `LogisticRegression(C=5.2)`).


In [ ]:
from sklearn.naive_bayes import GaussianNB

from sklearn.tree import DecisionTreeClassifier

In [ ]:
lr_clf = LogisticRegression()
#lr_clf = DecisionTreeClassifier()
lr_clf.fit(train_features, train_labels)

LogisticRegression()

In [ ]:
ypred = lr_clf.predict(test_features)

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
f1_score(ypred, test_labels,average='weighted')

0.583596214511041

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(test_labels, ypred))

              precision    recall  f1-score   support

           0       0.41      1.00      0.58       185
           1       0.00      0.00      0.00       129
           2       0.00      0.00      0.00       107
           3       0.00      0.00      0.00        28

    accuracy                           0.41       449
   macro avg       0.10      0.25      0.15       449
weighted avg       0.17      0.41      0.24       449



In [ ]:
dt_clf = DecisionTreeClassifier(max_depth=2)
dt_clf.fit(train_features, train_labels)
dt_predictions = dt_clf.predict(test_features)


In [ ]:
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(test_labels, dt_predictions)
print("Accuracy:", accuracy)

# You can also print a classification report for more detailed metrics
print(classification_report(test_labels, dt_predictions))




Accuracy: 0.41202672605790647
              precision    recall  f1-score   support

           0       0.41      1.00      0.58       185
           1       0.00      0.00      0.00       129
           2       0.00      0.00      0.00       107
           3       0.00      0.00      0.00        28

    accuracy                           0.41       449
   macro avg       0.10      0.25      0.15       449
weighted avg       0.17      0.41      0.24       449



In [ ]:
f1_score(dt_predictions, test_labels,average='weighted')

0.583596214511041

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(n_estimators=200, max_depth=3)
rf_clf.fit(train_features, train_labels)
rf_predictions = rf_clf.predict(test_features)
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(test_labels, rf_predictions)
print("Accuracy:", accuracy)

# You can also print a classification report for more detailed metrics
print(classification_report(test_labels, rf_predictions))



Accuracy: 0.42316258351893093
              precision    recall  f1-score   support

           0       0.42      0.97      0.58       185
           1       0.00      0.00      0.00       129
           2       0.52      0.10      0.17       107
           3       0.00      0.00      0.00        28

    accuracy                           0.42       449
   macro avg       0.24      0.27      0.19       449
weighted avg       0.30      0.42      0.28       449



In [ ]:
f1_score(rf_predictions, test_labels,average='weighted')

0.5647371061121869

In [ ]:
from sklearn.ensemble import VotingClassifier

# Create a VotingClassifier with your individual models
voting_clf = VotingClassifier(estimators=[
    ('lr', lr_clf),  # Replace lr_clf with your trained Logistic Regression model
    ('rf', rf_clf),  # Replace rf_clf with your trained Random Forest model
    ('dt', dt_clf)   # Replace dt_clf with your trained DecisionTreeClassifier model
], voting='hard')  # 'hard' for majority vote, or 'soft' for weighted vote

# Fit the VotingClassifier on your training data (optional)
voting_clf.fit(train_features, train_labels)
ensemble_predictions = voting_clf.predict(test_features)


In [ ]:
from sklearn.metrics import accuracy_score, f1_score

ensemble_accuracy = accuracy_score(test_labels, ensemble_predictions)
ensemble_f1 = f1_score(test_labels, ensemble_predictions, average='weighted')

print("Ensemble Accuracy:", ensemble_accuracy)
print("Ensemble F1-score:", ensemble_f1)


Ensemble Accuracy: 0.41202672605790647
Ensemble F1-score: 0.24045723760477195
